## Python 爬蟲練習：自Goodinfo!抓取台灣半導體上市公司之當日PBR
步驟摘要:
1. 建立股票代號清單
2. 連線[Goodinfo!](https://goodinfo.tw/StockInfo/index.asp)並抓取HTML。（使用request套件）
3. 解析抓下的HTML檔，從中找出PBR資料，並保留到空白清單中（使用BeautifulSoup4套件）
4. 連線到MySQL
5. 在MySQL建立本日的空白表格
6. 將剛剛抓取的PBR資料存入MySQL中的本日表格

爬蟲教學請參考下面影片
[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/9Z9xKWfNo7k/0.jpg)](https://www.youtube.com/watch?v=9Z9xKWfNo7k)  


---  
其他說明  
* 大部分的網站都會鎖爬蟲程式，所以我用fake-useragent套件，建立了虛擬的User Agent共十組，來模仿這是10個不同裝置登入的樣子，避免同一台裝置因短時間內大量下載資料而被鎖住。但其實此練習僅抓72筆資料，算很小的下載量，此步驟可略過。感謝[Goodinfo!](https://goodinfo.tw/StockInfo/index.asp)提供完整且免費的資料啊～    
  
    
* 另外則是連線方式，因為[Goodinfo!](https://goodinfo.tw/StockInfo/index.asp)採HTTP POST，因此用request連線需要作一下處理。更多關於HTTP連線方式可參考 [淺談 HTTP Method：表單中的 GET 與 POST 有什麼差別？](https://blog.toright.com/posts/1203/%E6%B7%BA%E8%AB%87-http-method%EF%BC%9A%E8%A1%A8%E5%96%AE%E4%B8%AD%E7%9A%84-get-%E8%88%87-post-%E6%9C%89%E4%BB%80%E9%BA%BC%E5%B7%AE%E5%88%A5%EF%BC%9F.html)。我只是業餘的，靠google大法來處理每次出現的Bug，還是得感謝各位無私分享知識的高手們。    
 
   
   
* 關於Python怎們跟MySQL溝通，我是使用MySQL個官方DB-API叫做mysql-connector-python，如何安裝跟簡易的操作請參考我的[這篇](https://thisisyuwang.blogspot.com/2020/02/jupyter-notebookmysql-databasemysql.html)。

*下面的code可存成.py檔，寫一個shell丟桌面。下班開電腦，點兩下就可以執行。*

In [ ]:
#建立股票清單(台股半導體代號)
num= [2303, 2329, 2330, 2337, 2338, 2342, 2344, 2351, 2363, 2369, 2379,2388, 2401, 2408, 
      2434, 2436, 2441, 2449, 2451, 2454, 2458, 2481,3006, 3014, 3016, 3034, 3035, 3041, 
      3054, 3094, 3189, 3257, 3413,3443, 3530, 3532, 3536, 3545, 3583, 3588, 3661, 3686, 
      3711, 4919,4952, 4961, 4967, 4968, 5269, 5285, 5305, 5471, 6202, 6239, 6243, 6257, 
      6271, 6415, 6451, 6525, 6531, 6533, 6552, 6573, 8016, 8028,8081, 8110, 8131, 8150, 
      8261, 8271]
#載入必要模組
import requests #傳送request
import bs4 as bs #beautifulsoup4 解HTML用
import pandas as pd
import numpy as np


#建立空表格
df = pd.DataFrame({"Stock":num,"PBR":np.zeros(len(num))})
df.set_index("Stock", inplace= True)

##建立虛擬的Header User agent清單,防止IP被鎖。可用fake-useragent套件創建隨機user agent
ua=['Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1468.0 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36',
 'Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.124 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
 'Mozilla/5.0 (X11; OpenBSD i386) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36',
 'Mozilla/5.0 (Windows; U; Windows NT 6.1; de-DE) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36']
#開始下載資料
try:
    for n in range(0,len(num)):
        PBR_Url = "https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=%i" % num[n] #連線goodinfo
        header=ua[np.random.randint(0,10)]  #亂數抽一個user agent
        headerForReq={"User-Agent":header} #r建立dict: request header
        response_goodinfo = requests.post(PBR_Url, headers=headerForReq) #
        result = bs.BeautifulSoup(response_goodinfo.content, "html.parser")#提供 request header
        data = result.find_all("td")  #從抓取的HTML檔中，找出有<td>標籤, 建立名為data的list
        if data[102].string != "N/A": #arg 102 或103 (此公司沒有PER資料時會多一個<td>標籤) 是PBR在data中的位置  
            PBR = float(data[102].string)
            df.loc[num[n]]=PBR
            print('Stock %i' % num[n], 'PBR :', PBR, ",第",n+1,"/",len(num),"筆")
        elif data[102].string == "N/A":
            PBR = float(data[103].string)
            df.loc[num[n]]=PBR
            print('Stock %i' % num[n], 'PBR :', PBR, ",第",n+1,"/",len(num),"筆")
        else:  
            print('Stock %i' % num[n],"got ERROR")
except IndexError:
    print("檢查",header, ", 這個可能不能用。")
    print('或者IP可能又被鎖')

    
#連線資料庫        
import mysql.connector
HugoDB = mysql.connector.connect(
  host = "127.0.0.1",  # Input the database server IP
  user = "root", #Input the username
  password = "password", #input the username
  database = "MyDatabase", #input the name of the database
  )
cursor=HugoDB.cursor() 
#建立當日表格
import datetime as dt
today = str(dt.date.today().strftime("%Y_%m_%d"))#生成今日日期標籤 output_ex: 2099_12_31
CreateTableCmd="CREATE TABLE IF NOT EXISTS %s_TW_SEMICOND_PBR(Date DATE,Stock INT NOT NULL DEFAULT '0',PBR FLOAT(7,3) NOT NULL DEFAULT '0')ENGINE=MyISAM DEFAULT CHARSET=big5;"%today  #mysql建立新表格的指令
DropTableCmd="DROP TABLE IF EXISTS %s_TW_SEMICOND_PBR;"%today #刪除重複mysql建立新表格的指令
TableToday="%s_TW_SEMICOND_PBR"%today #今日表格 output_ex: 2099_12_31_TW_SEMICOND_PBR
cursor.execute(DropTableCmd)
cursor.execute(CreateTableCmd)

#建立資料清單
inputdata = []
for i in np.arange(0,len(df.index)):
    stock = df.index[i]
    inputdata.append((str(stock),str(float(df.loc[stock]))));  #全部轉成string格式, 防止放入cursor.excutemany()時出錯

#將當日資料存入MySQL    
sqlcmd="INSERT INTO "+TableToday +" (Date,Stock,PBR) VALUES (DATE(NOW()),%s, %s);"  #mysql command
cursor.executemany(sqlcmd,inputdata)
HugoDB.commit()    

print("已存入本日PBR")